# Mount the S3 Bucket containing data from Kafka
1. Get the AWS authentication key file 
2. Mount the S3 bucket
3. Create 3 dataframes from the 3 locations in the mounted bucket
4. Copy Dataframes to Global Temporary Views


## Get authentication File

In [0]:
%run "./AWS Access Utils"

## Mount the bucket

In [0]:
# Mount name for the bucket
MOUNT_NAME = "/mnt/user-129a67850695-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
try:
    dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
except:
# If error try unmounting it first
    dbutils.fs.unmount(MOUNT_NAME)
    dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

display(dbutils.fs.ls("/mnt/user-129a67850695-bucket/"))

In [0]:
# We need to turn off delta format check before trying to read the json files because of a change to Databricks configuration
spark.conf.set("spark.databricks.delta.formatCheck.enabled", False)

## Read pin data to a dataframe

In [0]:
# Read from S3
df_pin = read_from_S3("pin", mount=True)

## Read geo data to a dataframe

In [0]:
# Read from S3
df_geo = read_from_S3("geo", mount=True)

## Read user data to a dataframe

In [0]:
# Read from S3
df_user = read_from_S3("user", mount=True)

## Copy dataframes to global temporary views

In [0]:
df_pin.createOrReplaceGlobalTempView("gtv_129a67850695_pin")
df_geo.createOrReplaceGlobalTempView("gtv_129a67850695_geo")
df_user.createOrReplaceGlobalTempView("gtv_129a67850695_user")